# Lecture 16

Continuing from lecture 15

In [3]:
import numpy as np
import math

# Create some virtual classes
class base:
    __name=""
    
    def __init__(self,name):
        self.__name=name

    def name(self):
        return self.__name
    

class data(base):
    def __init__(self,name):
        base.__init__(self,name)
        
class alg(base):
    def __init__(self,name):
        base.__init__(self,name)

In [4]:
class grade(data):
    __value=0
    __numerical=True
    __gradebook_name=str()
    __letter_grades=["F-","F","F+","D-","D","D+","C-","C","C+","B-","B","B+","A-","A","A+"]
    
    def __init__(self,name,numerical=True,value=None):
        if value:
            if isinstance(value,(int,float)):
                self.__numerical=True
            elif isinstance(value,str):
                self.__numerical=False
            self.set(value)
        else:            
            self.__numerical=numerical
        self.__gradebook_name=name
        data.__init__(self,name+" Grade Algorithm")        

    def set(self,value):
        if isinstance(value,(int,float)) and self.__numerical:
            self.__value=value
        elif isinstance(value,str) and not self.__numerical:
            if value in self.__letter_grades:
                self.__value=value
        else:
            print self.name()+" Error: Bad Grade."
            raise Exception
    
    def value(self):
        return self.__value
    
    def numerical(self):
        return self.__numerical
    
    def gradebook_name(self):
        return self.__gradebook_name
    
    def __str__(self):
        return self.__gradebook_name+": "+str(self.__value)

class student(data):
    __id_number=0
    __grades=dict()
    
    def __init__(self,first_name, last_name,id_number):
        self.__id_number=id_number
        self.__grades=dict()
        data.__init__(self,first_name+" "+last_name+" Student Data")

    def add_grade(self,a_grade,overwrite=False):
        if overwrite or not a_grade.gradebook_name() in self.__grades:
            self.__grades[a_grade.gradebook_name()]=a_grade
        else:
            print self.name()+" Error Adding Grade "+a_grade.name()+". Grade already exists."
            raise Exception

    def id_number(self):
        return self.__id_number
    
    def __getitem__(self,key):
        return self.__grades[key]
    
    def print_grades(self):
        for grade in self.__grades:
            print self.__grades[grade]
    


## Algorithms

In the previous lecture, we ended up with three different types of algorithms:

* `grade_calcuator` algorithms: Take grade from the student and return another grade. We use this for assigning letter grades and curving.
* `statistics_calculator` algorithms: Take a set of grades for all students and return numbers associated with the class. We used these to compute the mean and standard deviation for a class.
* `summary_calculator` algorithms: Take many grades from a student and return a another grade. We used these to sum grades of a student.

Here are the base classes:

In [5]:
class grade_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade):
        raise NotImplementedError
        # Returns a grade
    
class statistics_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,grades):
        raise NotImplementedError
        # returns numbers
                
class summary_calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_student):
        raise NotImplementedError
        # returns a grade
        

While this approach works, users will need to be aware of how to use each type of algorithm and it would be difficult to create a system that would automate tasks, like curving a grade, that would require running several algorithms. We may be able to find a more general implementation that can unify all of these types of algorithms. There are several issues to consider:

* **Input**: Each of these algorithms take different inputs, either a single grade, all grades of a specific type for all students, or different grades for the same student.
* **Output**: Some of these algorithms generate new grades for each student, while others provide compute  over the whole class.
* **Information storage**: While we can store grades with students, other information like means and standard deviation have to do with the class and not any individual student. 

Lets first address the information storage issue. We simply need a mechanism to store information that has to do with a class instead of a student. The gradebook is what holds all of the students in the class, so we just add new data members to keep additional information. The only type of information that we are holding for now are specific exam means and standard deviations. We could choose to create new data objects for such information. Or we could simply stuff the information into a dictionary. The latter is easier and may suffice, so let's try it.


In [6]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number]=a_student
        
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print self.name()+" Error: Did not find student."
            
    def apply_grader(self,a_grader,grade_name):
        for k,a_student in self.__students.iteritems():
            a_student.add_grade(a_grader.apply(a_student[grade_name]))
            
    def apply_stats(self,a_stat_comp,grade_name):
        grades=list()
        for k,a_student in self.__students.iteritems():
            grades.append(a_student[grade_name].value())
            
        return a_stat_comp.apply(grades)
        
  

Let's test:

In [7]:
a_grade_book=grade_book("Test Gradebook")
a_grade_book["Example Data"]=100
print a_grade_book["Example Data"]

100


Now lets address the input/output issue. We have several options:

1. Keep things the same. We keep 3 different types of algorithms. Note that necessarily needed to create separate apply methods in the `grade_book` class (`apply_grader`, `apply_stats`, ...) for each type of algorithm with a different method name. Here the user has to know what type of algorithm he/she is applying and then use the right apply method.

2. Keep the 3 differnt types of algorithms, but create a general apply method to `grade_book` that checks the type of the algorithm and then uses the right apply method.

3. Unify the 3 types of algorithms, pass into them the `grade_book` instance and let them decide what to do.

Option 2 is a reasonable solution, but let's try option 3 because it provides more flexibility for future algorithms that may take other input/output combinations. 

First turn our three different `calculator` algorithm base classes into one:

In [23]:
class calculator(alg):    
    def __init__(self,name):
        alg.__init__(self,name)

    def apply(self,a_grade_book):
        raise NotImplementedError


Next, lets combine the `apply` methods in the `grade_book` class:

In [11]:
class grade_book(data):
    # New member class to hold arbitrary data associated with the class

    __data=dict()
    __students=dict()
    
    def __init__(self,name):
        data.__init__(self,name+" Course Grade Book")
        self.__students=dict()
        self.__data=dict()
        
    # New method to access data
    def __getitem__(self,key):
        return self.__data[key]
            
    # New method to add data
    def __setitem__(self, key, value):
        self.__data[key] = value
        
    def add_student(self,a_student):
        self.__students[a_student.id_number()]=a_student

    # New method to allow iterating over students
    def get_students(self):
        return self.__students
    
    def assign_grade(self,key,a_grade):
        the_student=None
        try:
            the_student=self.__students[key]
        except:
            for id in self.__students:
                if key == self.__students[id].name():
                    the_student=self.__students[id]
                    break
        if the_student:
            the_student.add_grade(a_grade)
        else:
            print self.name()+" Error: Did not find student."
            
    def apply_calculator(self,a_calculator,**kwargs):
        a_calculator.apply(self,**kwargs)
        
    

Now we have to move the logic of what each calculator algorithm uses into the algorithm. For example: 

In [12]:
class uncurved_letter_grade_percent(calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,max_grade=100.):
        self.__max_grade=max_grade
        self.__grade_name=grade_name
        calculator.__init__(self,
                                  "Uncurved Percent Based Grade Calculator "+self.__grade_name+" Max="+str(self.__max_grade))
        
    def apply(self,a_grade_book,grade_name=None,**kwargs):
        if grade_name:
            pass
        else:
            grade_name=self.__grade_name
            
  
        for k,a_student in a_grade_book.get_students().iteritems():
            a_grade=a_student[grade_name]

            if not a_grade.numerical():
                print self.name()+ " Error: Did not get a numerical grade as input."
                raise Exception
    
            percent=a_grade.value()/self.__max_grade
        
            for i,v in enumerate(self.__grades_definition):
                #print percent, i, v
                if percent>=v[0]:
                    break
                            
            a_student.add_grade(grade(grade_name+" Letter",value=self.__grades_definition[i][1]))
            

Let's test with our class data:

In [20]:
import pandas as pd
Data=pd.read_csv("Scores.csv")

a_grade_book=grade_book("Data 1401")

for student_i in range(Data.shape[0]):
    a_student_0=student("Student",str(student_i),student_i)

    for k in Data.keys():
        a_student_0.add_grade(grade(k,value=Data[k][student_i]))

    a_grade_book.add_student(a_student_0)
        

In [21]:
a_grade_book.apply_calculator(uncurved_letter_grade_percent("l2_2",max_grade=10))
for k,a_student in a_grade_book.get_students().iteritems():
    print a_student.id_number(),a_student["l2_2"],a_student["l2_2 Letter"]

0 l2_2: 10 l2_2 Letter: A+
1 l2_2: 10 l2_2 Letter: A+
2 l2_2: 10 l2_2 Letter: A+
3 l2_2: 10 l2_2 Letter: A+
4 l2_2: 10 l2_2 Letter: A+
5 l2_2: 10 l2_2 Letter: A+
6 l2_2: 10 l2_2 Letter: A+
7 l2_2: 10 l2_2 Letter: A+
8 l2_2: 0 l2_2 Letter: F-
9 l2_2: 10 l2_2 Letter: A+
10 l2_2: 10 l2_2 Letter: A+
11 l2_2: 10 l2_2 Letter: A+
12 l2_2: 10 l2_2 Letter: A+
13 l2_2: 10 l2_2 Letter: A+
14 l2_2: 10 l2_2 Letter: A+
15 l2_2: 0 l2_2 Letter: F-


In [ ]:
a_grade_book.get_students()

## Stats Computation

In [28]:
class mean_std_calculator(calculator):
    def __init__(self):
        calculator.__init__(self,"Mean and Standard Deviation Calculator")
        
    def apply(self,a_grade_book,grade_name,**kwargs):
        grades=list()
        for k,a_student in a_grade_book.get_students().iteritems():
            grades.append(a_student[grade_name].value())
        
        a_grade_book[grade_name+" Mean"] = np.mean(grades)
        a_grade_book[grade_name+" STD"] = math.sqrt(np.var(grades))


In [30]:
a_grade_book.apply_calculator(mean_std_calculator(),grade_name="l3_2")
print a_grade_book["l3_2 Mean"]

7.78125


## Grade Summing

In [ ]:

class grade_summer(summary_calculator):
    def __init__(self,prefix,n):
        self.__prefix=prefix
        self.__n=n
        statistics_calculator.__init__(self,"Sum Grades")
        
    def apply(self,a_student):
        labels=[prefix+str(x) for x in range(1,n)]
        
        grade_sum=0.
        for label in labels:
            grade_sum+=a_student[label]
            
        a_student.add_grade(grade(prefix+"sum",value=grade_sum))

## Curved Letter Grade

In [ ]:
class curved_letter_grade(grade_calculator):
    __grades_definition=[ (.97,"A+"),
                          (.93,"A"),
                          (.9,"A-"),
                          (.87,"B+"),
                          (.83,"B"),
                          (.8,"B-"),
                          (.77,"C+"),
                          (.73,"C"),
                          (.7,"C-"),
                          (.67,"C+"),
                          (.63,"C"),
                          (.6,"C-"),
                          (.57,"F+"),
                          (.53,"F"),
                          (0.,"F-")]
    __max_grade=100.
    __grade_name=str()
    
    def __init__(self,grade_name,mean,std,max_grade=100.):
        self.__max_grade=max_grade
        self.__mean=mean
        self.__std=std
        self.__grade_name=grade_name
        grade_calculator.__init__(self,
                                  "Curved Percent Based Grade Calculator "+self.__grade_name+ \
                                  " Mean="+str(self.__mean)+\
                                  " STD="+str(self.__std)+\
                                  " Max="+str(self.__max_grade))
        

    def apply(self,a_grade):
        if not isinstance(a_grade,grade):
            print isinstance(a_grade,grade)
            print self.name()+ " Error: Did not get an proper grade as input."
            raise Exception
        if not a_grade.numerical():
            print self.name()+ " Error: Did not get a numerical grade as input."
            raise Exception
    
        # Rescale the grade
        percent=a_grade.value()/self.__max_grade
        shift_to_zero=percent-(self.__mean/self.__max_grade)
        scale_std=0.1*shift_to_zero/(self.__std/self.__max_grade)
        scaled_percent=scale_std+0.8
        
        for i,v in enumerate(self.__grades_definition):
            #print percent, i, v
            if scaled_percent>=v[0]:
                break
                            
        return grade(self.__grade_name,value=self.__grades_definition[i][1])
            